# 05 - Continuous Training

After testing, compiling, and uploading the pipeline definition to Cloud Storage, the pipeline is executed with respect to a trigger. We use [Cloud Functions](https://cloud.google.com/functions) and [Cloud Pub/Sub](https://cloud.google.com/pubsub) as a triggering mechanism. The triggering can be scheduled using [Cloud Schedular](https://cloud.google.com/scheduler). The trigger source sends a message to a Cloud Pub/Sub topic that the Cloud Function listens to, and then it submits the pipeline to AI Platform Managed Pipelines to be executed.

This notebook covers the following steps:
1. Create the Cloud Pub/Sub topic.
2. Deploy the Cloud Function 
3. Test triggering a pipeline.
4. Extracting pipeline run metadata.

## Setup

### Import libraries

In [1]:
import json
import os
import logging
import tensorflow as tf
import tfx
import IPython 

logging.getLogger().setLevel(logging.INFO)

print("Tensorflow Version:", tfx.__version__)

2021-09-01 09:17:39.993723: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


Tensorflow Version: 1.2.0


### Setup Google Cloud project

In [2]:
PROJECT = 'aiops-industrialization' # Change to your project id.
REGION = 'us-central1' # Change to your region.
BUCKET =  'aiops-industrialization-bucket-ravi'#'[your-bucket-name]' # Change to your bucket name.

if PROJECT == "" or PROJECT is None or PROJECT == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT = shell_output[0]
    
if BUCKET == "" or BUCKET is None or BUCKET == "[your-bucket-name]":
    # Get your bucket name to GCP projet id
    BUCKET = PROJECT

print("Project ID:", PROJECT)
print("Region:", REGION)
print("Bucket name:", BUCKET)

Project ID: aiops-industrialization
Region: us-central1
Bucket name: aiops-industrialization-bucket-ravi


### Set configurations

In [11]:
#VERSION = 'v01'
VERSION = ''
DATASET_DISPLAY_NAME = 'chicago-taxi-tips'
MODEL_DISPLAY_NAME = f'{DATASET_DISPLAY_NAME}-classifier-{VERSION}'
PIPELINE_NAME = f'{MODEL_DISPLAY_NAME}train-pipeline'

PIPELINES_STORE = f'gs://{BUCKET}/{DATASET_DISPLAY_NAME}/compiled_pipelines/'
GCS_PIPELINE_FILE_LOCATION = os.path.join(PIPELINES_STORE, f'{PIPELINE_NAME}.json')
PUBSUB_TOPIC = f'trigger-{PIPELINE_NAME}'
CLOUD_FUNCTION_NAME = f'trigger-{PIPELINE_NAME}-fn'

In [19]:
print(GCS_PIPELINE_FILE_LOCATION)
print(PUBSUB_TOPIC)
print(CLOUD_FUNCTION_NAME)
#gs://aiops-industrialization-bucket-ravi/chicago-taxi-tips/compiled_pipelines/chicago-taxi-tips-classifier-train-pipeline.json

gs://aiops-industrialization-bucket-ravi/chicago-taxi-tips/compiled_pipelines/chicago-taxi-tips-classifier-train-pipeline.json
trigger-chicago-taxi-tips-classifier-train-pipeline
trigger-chicago-taxi-tips-classifier-train-pipeline-fn


In [13]:
!gsutil ls {GCS_PIPELINE_FILE_LOCATION}

gs://aiops-industrialization-bucket-ravi/chicago-taxi-tips/compiled_pipelines/chicago-taxi-tips-classifier-train-pipeline.json


## 1. Create a Pub/Sub topic

In [16]:
!gcloud pubsub topics create {PUBSUB_TOPIC}

Created topic [projects/aiops-industrialization/topics/trigger-chicago-taxi-tips-classifier-train-pipeline].


## 2. Deploy the Cloud Function

In [17]:
ENV_VARS=f"""\
PROJECT={PROJECT},\
REGION={REGION},\
GCS_PIPELINE_FILE_LOCATION={GCS_PIPELINE_FILE_LOCATION}
"""

!echo {ENV_VARS}

PROJECT=aiops-industrialization,REGION=us-central1,GCS_PIPELINE_FILE_LOCATION=gs://aiops-industrialization-bucket-ravi/chicago-taxi-tips/compiled_pipelines/chicago-taxi-tips-classifier-train-pipeline.json


In [18]:
!rm -r src/pipeline_triggering/.ipynb_checkpoints

rm: cannot remove 'src/pipeline_triggering/.ipynb_checkpoints': No such file or directory


In [20]:
!gcloud functions deploy {CLOUD_FUNCTION_NAME} \
    --region={REGION} \
    --trigger-topic={PUBSUB_TOPIC} \
    --runtime=python37 \
    --source=src/pipeline_triggering\
    --entry-point=trigger_pipeline\
    --stage-bucket={BUCKET}\
    --update-env-vars={ENV_VARS}

Deploying function (may take a while - up to 2 minutes)...⠹                    
For Cloud Build Logs, visit: https://console.cloud.google.com/cloud-build/builds;region=us-central1/c5e63dfb-fc64-4bcb-8ecc-7439ff8e5289?project=175728527123
Deploying function (may take a while - up to 2 minutes)...done.                
availableMemoryMb: 256
buildId: c5e63dfb-fc64-4bcb-8ecc-7439ff8e5289
buildName: projects/175728527123/locations/us-central1/builds/c5e63dfb-fc64-4bcb-8ecc-7439ff8e5289
entryPoint: trigger_pipeline
environmentVariables:
  GCS_PIPELINE_FILE_LOCATION: gs://aiops-industrialization-bucket-ravi/chicago-taxi-tips/compiled_pipelines/chicago-taxi-tips-classifier-train-pipeline.json
  PROJECT: aiops-industrialization
  REGION: us-central1
eventTrigger:
  eventType: google.pubsub.topic.publish
  failurePolicy: {}
  resource: projects/aiops-industrialization/topics/trigger-chicago-taxi-tips-classifier-train-pipeline
  service: pubsub.googleapis.com
ingressSettings: ALLOW_ALL
labels:
  

In [21]:
cloud_fn_url = f"https://console.cloud.google.com/functions/details/{REGION}/{CLOUD_FUNCTION_NAME}"
html = f'See the Cloud Function details <a href="{cloud_fn_url}" target="_blank">here</a>.'
IPython.display.display(IPython.display.HTML(html))

## 3. Trigger the pipeline

In [31]:
from google.cloud import pubsub

publish_client = pubsub.PublisherClient()
topic = f'projects/{PROJECT}/topics/{PUBSUB_TOPIC}'
data = {
    'num_epochs': 7,
    'learning_rate': 0.0015,
    'batch_size': 512,
    'hidden_units': '256,126'
}
message = json.dumps(data)

_ = publish_client.publish(topic, message.encode())

Wait for a few seconds for the pipeline run to be submitted, then you can see the run in the Cloud Console

In [30]:
from kfp.v2.google.client import AIPlatformClient

pipeline_client = AIPlatformClient(
    project_id=PROJECT, region=REGION)
 
job_display_name = pipeline_client.list_jobs()['pipelineJobs'][0]['displayName']
job_url = f"https://console.cloud.google.com/vertex-ai/locations/{REGION}/pipelines/runs/{job_display_name}"
html = f'See the Pipeline job <a href="{job_url}" target="_blank">here</a>.'
IPython.display.display(IPython.display.HTML(html))

/home/jupyter/.local/lib/python3.7/site-packages/kfp/v2/google/client/client.py:175: FutureWarning: AIPlatformClient will be deprecated in v1.9. Please use PipelineJob https://googleapis.dev/python/aiplatform/latest/_modules/google/cloud/aiplatform/pipeline_jobs.html in Vertex SDK. Install the SDK using "pip install google-cloud-aiplatform"
  category=FutureWarning,


## 4. Extracting pipeline runs metadata

In [28]:
from google.cloud import aiplatform as vertex_ai

pipeline_df = vertex_ai.get_pipeline_df(PIPELINE_NAME)
pipeline_df = pipeline_df[pipeline_df.pipeline_name == PIPELINE_NAME]
pipeline_df.T

,0,1,2
pipeline_name,chicago-taxi-tips-classifier-train-pipeline,chicago-taxi-tips-classifier-train-pipeline,chicago-taxi-tips-classifier-train-pipeline
run_name,chicago-taxi-tips-classifier-train-pipeline-20...,chicago-taxi-tips-classifier-train-pipeline-20...,chicago-taxi-tips-classifier-train-pipeline-20...
param.input:learning_rate,0.0015,0.0015,0.003
param.input:num_epochs,7,7,30
param.input:hidden_units,"256,126","256,126","128,128"
param.input:batch_size,512,512,512
